In [1]:
from datasets import load_dataset
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats

# ------------------------------------------------------
# 1) Load dataset (train / validation / test)
# ------------------------------------------------------
dataset = load_dataset("AliFartout/PEYMA-ARMAN-Mixed")

train_data = dataset["train"]
val_data   = dataset["validation"]
test_data  = dataset["test"]

# ------------------------------------------------------
# 2) Feature extraction for CRF
# ------------------------------------------------------
def word2features(tokens, i):
    word = tokens[i]
    features = {
        "bias": 1.0,
        "word.lower()": word.lower(),
        "word.isupper()": word.isupper(),
        "word.istitle()": word.istitle(),
        "word.isdigit()": word.isdigit(),
    }

    # previous token
    if i > 0:
        prev = tokens[i - 1]
        features.update({
            "-1:lower": prev.lower(),
            "-1:istitle": prev.istitle(),
            "-1:isupper": prev.isupper(),
        })
    else:
        features["BOS"] = True

    # next token
    if i < len(tokens) - 1:
        nxt = tokens[i + 1]
        features.update({
            "+1:lower": nxt.lower(),
            "+1:istitle": nxt.istitle(),
            "+1:isupper": nxt.isupper(),
        })
    else:
        features["EOS"] = True

    return features


def extract_features(split):
    return [[word2features(sample["tokens"], i)
             for i in range(len(sample["tokens"]))] 
            for sample in split]


def get_labels(split):
    return [sample["ner_tags_names"] for sample in split]


# ------------------------------------------------------
# 3) Prepare features + labels
# ------------------------------------------------------
X_train = extract_features(train_data)
y_train = get_labels(train_data)

X_val = extract_features(val_data)
y_val = get_labels(val_data)

X_test = extract_features(test_data)
y_test = get_labels(test_data)

# ------------------------------------------------------
# 4) Train CRF model 
# ------------------------------------------------------
crf = CRF(
    algorithm="lbfgs",
    max_iterations=200,
    all_possible_transitions=True
)

# Hyperparameter search space
params_space = {
    "c1": scipy.stats.expon(scale=0.5),   # L1 regularization
    "c2": scipy.stats.expon(scale=0.05),  # L2 regularization
}

# Use validation set for tuning
rs = RandomizedSearchCV(
    crf,
    params_space,
    cv=[(range(len(X_train)), range(len(X_train), len(X_train)+len(X_val)))],  # manual split
    verbose=1,
    n_iter=10,
    n_jobs=-1,
)


# Combine train + val for proper cross-validation
X_train_full = X_train + X_val
y_train_full = y_train + y_val
crf.fit(X_train, y_train)

print("🔍 Running Hyperparameter Search...")
rs.fit(X_train_full, y_train_full)

# Best model
best_crf = rs.best_estimator_
print("Best hyperparameters:", rs.best_params_)

y_pred = best_crf.predict(X_test)
print(flat_classification_report(y_test, y_pred))



🔍 Running Hyperparameter Search...
Fitting 1 folds for each of 10 candidates, totalling 10 fits
Best hyperparameters: {'c1': np.float64(0.04730271693953093), 'c2': np.float64(0.011604851603186507)}
              precision    recall  f1-score   support

       B_DAT       0.75      0.48      0.59       179
       B_EVE       0.95      0.97      0.96       218
       B_FAC       0.92      0.96      0.94       124
       B_LOC       0.91      0.86      0.88      1855
       B_MON       0.92      0.69      0.79        51
       B_ORG       0.90      0.85      0.87      2010
       B_PCT       0.91      0.74      0.82        27
       B_PER       0.96      0.86      0.91      1558
       B_PRO       0.97      0.95      0.96       281
       B_TIM       0.76      0.48      0.59        27
       I_DAT       0.77      0.50      0.60       235
       I_EVE       0.96      0.99      0.98       697
       I_FAC       0.92      0.98      0.95       237
       I_LOC       0.78      0.75      0.76  

In [2]:
from seqeval.metrics import (
    f1_score, precision_score, recall_score,
    classification_report
)



# ------------------------------------------------------
# Micro / Macro / Weighted scores
# ------------------------------------------------------
print("Micro F1:", f1_score(y_test, y_pred, average='micro'))
print("Macro F1:", f1_score(y_test, y_pred, average='macro'))
print("Weighted F1:", f1_score(y_test, y_pred, average='weighted'))

print("\nMicro Precision:", precision_score(y_test, y_pred, average='micro'))
print("Macro Precision:", precision_score(y_test, y_pred, average='macro'))
print("Weighted Precision:", precision_score(y_test, y_pred, average='weighted'))

print("\nMicro Recall:", recall_score(y_test, y_pred, average='micro'))
print("Macro Recall:", recall_score(y_test, y_pred, average='macro'))
print("Weighted Recall:", recall_score(y_test, y_pred, average='weighted'))


c:\Users\fatim\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\fatim\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\fatim\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\fatim\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: I_PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\fatim\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 

Micro F1: 0.8710921473701485
Macro F1: 0.8193565486889633
Weighted F1: 0.8693932131613525

Micro Precision: 0.9062660064879631
Macro Precision: 0.883034474982637
Weighted Precision: 0.9047315799161992

Micro Recall: 0.8385466034755135
Macro Recall: 0.77373418920357
Weighted Recall: 0.8385466034755135
